In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("csv_pca/psam_p06.csv")
df.head()

In [ ]:
len(df)

In [ ]:
len(np.unique(df.PUMA.values))

In [ ]:
INCOME_COL = "PINCP"
AGE_COL = "AGEP"

In [ ]:
df = df[df[INCOME_COL] != 0.0]

In [ ]:
currdf = df[df["PUMA"] == 8103]
plt.scatter(currdf[AGE_COL], currdf[INCOME_COL])

currdf = df[df["PUMA"] == 3100]
plt.scatter(currdf[AGE_COL], currdf[INCOME_COL])

currdf = df[df["PUMA"] == 2901]
plt.scatter(currdf[AGE_COL], currdf[INCOME_COL])

currdf = df[df["PUMA"] == 2901]
plt.scatter(currdf[AGE_COL], currdf[INCOME_COL])

plt.yscale("log")

In [ ]:
avg_incomes = []
avg_ages = []
std_incomes = []
std_ages = []

for puma in np.unique(df.PUMA.values):
    currdf = df[df["PUMA"] == puma]
    incomes = currdf[INCOME_COL].values
    avg_incomes.append(np.nanmean(incomes))
    std_incomes.append(np.nanstd(incomes))
    ages = currdf[AGE_COL].values
    avg_ages.append(np.nanmean(ages))
    std_ages.append(np.nanstd(ages))

In [ ]:
plt.scatter(avg_ages, avg_incomes, color="r")
plt.title("AVG AGE vs AVG INCOME per PUMA")
plt.show()

In [ ]:
plt.scatter(std_ages, std_incomes)
plt.title("STD AGE vs STD INCOME per PUMA")
plt.show()

In [ ]:
import seaborn as sns
fig = plt.figure()
ax = fig.add_subplot(111)

for puma in np.unique(df.PUMA.values):
    currdf = df[df["PUMA"] == puma]
    incomes = currdf[INCOME_COL].values
    sns.kdeplot(np.log(incomes[incomes > 0]), ax=ax, alpha=0.3)

In [ ]:
import shapefile as shp

In [ ]:
sf = shp.Reader("csv_pca/ipums_puma_2010/ipums_puma_2010.shp")

plt.figure()
kept = 0
ids = []

for shape in sf.shapeRecords():
    if shape.record[4] != "California":
        continue
    
    kept += 1
    ids.append(int(shape.record[5]))
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    plt.plot(x,y)
plt.show()

In [ ]:
tmp = sf.shapeRecords()[0]

In [ ]:
tmp.record

In [ ]:
sorted(ids)

In [ ]:
np.unique(df.PUMA.values)

In [ ]:
import geopandas as gpd

shp = gpd.read_file("csv_pca/ipums_puma_2010/ipums_puma_2010.shp")

In [ ]:
shp = shp[shp["State"] == "California"]

In [ ]:
shp

In [ ]:
import geopandas as gpd

# open file
gdf = shp

# add NEIGHBORS column
gdf["NEIGHBORS"] = None  

for index, country in gdf.iterrows():   

    # get 'not disjoint' countries
    neighbors = gdf[~gdf.geometry.disjoint(country.geometry)].PUMA.tolist()

    # remove own name of the country from the list
    neighbors = [ name for name in neighbors if country.PUMA != name ]

    # add names of neighbors as NEIGHBORS value
    gdf.at[index, "NEIGHBORS"] = ", ".join(neighbors)


In [ ]:
shp

In [ ]:
puma2id = {x: i for i, x in enumerate(np.unique(df.PUMA.values))}

In [ ]:
np.random.seed(100)

income_data = []
for puma in np.unique(df.PUMA.values):
    currdf = df[df["PUMA"] == puma]
    inc = np.random.choice(currdf[INCOME_COL].values, 200)
    income_data.append(inc)

In [ ]:
import pickle

income_data = np.array(income_data)
with open("income_data/california_income_subsampled.pickle", "wb") as fp:
    pickle.dump(income_data, fp)

In [ ]:
n = len(np.unique(df.PUMA.values))

neighbors = np.zeros((n, n))
for index, puma in shp.iterrows():  
    row = puma2id[int(puma.PUMA)]
    curr_neigh = [int(x) for x in puma.NEIGHBORS.split(", ")]
    for neigh in curr_neigh:
        col = puma2id[neigh]
        neighbors[row, col] += 1

In [ ]:
with open("income_data/california_puma_neighbors.pickle", "wb") as fp:
    pickle.dump(neighbors, fp)